In [3]:
import time, geopandas as gpd
import rasterio
from rasterio.plot import reshape_as_image
from leafmap.download import download_naip
from pathlib import Path
import pandas as pd, geemap, ee, json
import rasterio
import numpy as np
from rasterio.windows import Window
import os
import geoai-py

ImportError: DLL load failed while importing _context: The specified module could not be found.

In [4]:
!conda create -n geoai python=3.10 -y
!conda activate geoai


'conda' is not recognized as an internal or external command,
operable program or batch file.
'conda' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
ee.Authenticate() # Authenticate Earth Engine
ee.Initialize() # Initialize the API
print("EE Version:", ee.__version__) # Print Earth Engine version

EE Version: 1.7.4


In [10]:
# Selects AOI: Mecklenburg County, NC
meck = (ee.FeatureCollection('TIGER/2018/Counties') #Loading the TIGER data 
        .filter(ee.Filter.eq('NAME', 'Mecklenburg'))
        .filter(ee.Filter.eq('STATEFP', '37'))) # ensures it’s the one in NC (state FIPS 37)

def add_props(img):
    yr = ee.Date(img.get('system:time_start')).get('year') # Extracts the acquisition year
    bands_str = img.bandNames().join(',')    # Stores the band list as a comma-separated string
    return img.set({'year': yr, 'bands_str': bands_str})

naip_all = (ee.ImageCollection('USDA/NAIP/DOQQ')  # Selects NAIP imagery
            .filterBounds(meck)
            .map(add_props))

years = ee.List(naip_all.aggregate_array('year')).distinct().sort(); print('Years available:', years.getInfo()) # Retrieves the list of available years

# Builds a small per-year table on the server
def summarize(y):
    icY = naip_all.filter(ee.Filter.eq('year', y))
    total   = icY.size()
    withN   = icY.filter(ee.Filter.listContains('system:band_names','N')).size()
    bandsets = icY.aggregate_array('bands_str').distinct().sort()
    return ee.Feature(None, {
        'year': y,
        'total_images': total,
        'images_with_N': withN,
        'band_sets': bandsets
    })
byYear_fc = ee.FeatureCollection(years.map(summarize))
rows = [f['properties'] for f in byYear_fc.getInfo()['features']]
summary_df = pd.DataFrame(rows).sort_values('year')
summary_df

Years available: [2004, 2005, 2006, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


,band_sets,images_with_N,total_images,year
0,"[R,G,B]",0,55,2004
1,"[R,G,B]",0,64,2005
2,"[R,G,B]",0,56,2006
3,"[R,G,B]",0,56,2008
4,"[R,G,B,N]",64,64,2009
5,"[R,G,B,N]",56,56,2010
6,"[R,G,B,N]",8,8,2011
7,"[R,G,B,N]",56,56,2012
8,"[R,G,B,N]",8,8,2013
9,"[R,G,B,N]",56,56,2014


In [11]:
YEAR = 2023

start = ee.Date.fromYMD(YEAR, 1, 1)
end = start.advance(1, 'year')

naip_year = (ee.ImageCollection('USDA/NAIP/DOQQ')
             .filterBounds(meck)
             .filterDate(start, end))

print('Image count:', naip_year.size().getInfo())

first = naip_year.first()
print('Band names of first image:', first.bandNames().getInfo())


Image count: 8
Band names of first image: ['R', 'G', 'B', 'N']


In [12]:
m = geemap.Map(center=[35.23, -80.84], zoom=10)

mosaic = naip_year.mosaic().clip(meck)
m.addLayer(mosaic, {'bands':['R','G','B'], 'min':0, 'max':255, 'gamma':1.1},
           f'NAIP RGB {YEAR}')
m

Map(center=[35.23, -80.84], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

In [13]:
# Reproject to a projected CRS (meters)
target_crs = "EPSG:26917"  # NAD83 / UTM zone 17N

mosaic = (
    naip_year
    .mosaic()
    .clip(meck)
    .select(['R','G','B','N'])   # ensure band order
    .reproject(crs=target_crs, scale=2)
)

In [14]:
#downloading to google drive

task = ee.batch.Export.image.toDrive(
    image=mosaic,
    description=f"NAIP_Meck_{YEAR}_2m",
    folder="GEE",
    fileNamePrefix=f"naip_meck_{YEAR}_2m",
    region=meck.geometry(),
    scale=2,
    crs=target_crs,
    maxPixels=1e13,
    fileFormat="GeoTIFF"
)

task.start()
print("Export started. Check Google Drive → GEE folder.")


Export started. Check Google Drive → GEE folder.


In [ ]:
while task.status()['state'] in ['READY', 'RUNNING']:
    print("Task state:", task.status()['state'])
    time.sleep(30)  # wait 30 seconds

print("Final task status:", task.status())

In [22]:
tif_path = r"H:/My Drive/GEE/naip_meck_2023_2m.tif"

with rasterio.open(tif_path) as src:
    print("CRS:", src.crs)
    print("Resolution:", src.res)
    print("Shape (bands, H, W):", src.count, src.height, src.width)
    print("Bounds:", src.bounds)


2025-12-15 12:23:27,700 - WARNING - CPLE_AppDefined in PROJ: proj_create_from_database: C:\Program Files\PostgreSQL\17\share\contrib\postgis-3.5\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 5 is expected. It comes from another PROJ installation.
2025-12-15 12:23:27,703 - WARNING - CPLE_AppDefined in The definition of projected CRS EPSG:26917 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.
2025-12-15 12:23:27,736 - INFO - GDAL signalled an error: err_no=1, msg='PROJ: proj_identify: C:\\Program Files\\PostgreSQL\\17\\share\\contrib\\postgis-3.5\\proj\\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 5 is expected. It comes from another PROJ installation.'


CRS: PROJCS["NAD83 / UTM zone 17N",GEOGCS["NAD83",DATUM["North American Datum 1983",SPHEROID["GRS 1980",6378137,298.257222101004]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-81],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Resolution: (2.0, 2.0)
Shape (bands, H, W): 4 28449 23118
Bounds: BoundingBox(left=494694.0, bottom=3873242.0, right=540930.0, top=3930140.0)


In [23]:
with rasterio.open(tif_path) as src:
    R = src.read(1).astype("float32")
    G = src.read(2)
    B = src.read(3)
    N = src.read(4)

ndvi = (N - R) / (N + R + 1e-6)

print("NDVI range:", np.nanmin(ndvi), np.nanmax(ndvi))

2025-12-15 12:28:06,422 - WARNING - CPLE_AppDefined in PROJ: proj_create_from_database: C:\Program Files\PostgreSQL\17\share\contrib\postgis-3.5\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 5 is expected. It comes from another PROJ installation.
2025-12-15 12:28:06,423 - WARNING - CPLE_AppDefined in The definition of projected CRS EPSG:26917 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.


NDVI range: -0.9896907 0.9907834


In [25]:
ndvi_path = r"H:/My Drive/GEE/naip_meck_2023_ndvi_2m.tif"

profile = src.profile
profile.update(count=1, dtype="float32")

with rasterio.open(ndvi_path, "w", **profile) as dst:
    dst.write(ndvi, 1)

2025-12-15 12:29:34,050 - INFO - GDAL signalled an error: err_no=1, msg='PROJ: proj_create_from_name: C:\\Program Files\\PostgreSQL\\17\\share\\contrib\\postgis-3.5\\proj\\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 5 is expected. It comes from another PROJ installation.'


In [ ]:
def tile_raster(
    in_raster,
    out_dir,
    tile_size=512,
    stride=512
):
    os.makedirs(out_dir, exist_ok=True)

    with rasterio.open(in_raster) as src:
        meta = src.meta.copy()
        width = src.width
        height = src.height

        tile_id = 0
        for y in range(0, height - tile_size, stride):
            for x in range(0, width - tile_size, stride):

                window = Window(x, y, tile_size, tile_size)
                transform = src.window_transform(window)

                tile = src.read(window=window)

                meta.update({
                    "height": tile_size,
                    "width": tile_size,
                    "transform": transform
                })

                out_path = os.path.join(out_dir, f"tile_{tile_id:05d}.tif")
                with rasterio.open(out_path, "w", **meta) as dst:
                    dst.write(tile)

                tile_id += 1

    print(f"Generated {tile_id} tiles.")


tile_raster(
    in_raster=tif_path,
    out_dir=r"H:/My Drive/GEE/tiles/naip_2023",
    tile_size=512,
    stride=512
)